In [1]:
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import copy

In [2]:
def rot90_t(arr,n):
    for i in range(n):
        arr = np.rot90(arr)
    return(arr)
# create color map
cmap = plt.cm.get_cmap('RdYlGn', 4)  # red, yellow, green, white
cmap.set_under('w')  # set color for values less than 0 to white

colors = [(1,1,1),(0, 0, 0), (0.5, 0.5, 0.5), (1, 0, 0), (0, 0, 1), (1, 0.5, 0)]
# Define the levels for each color
levels = [ 0 ,1, 2, 3, 4, 5]
item_map = ListedColormap(colors, N=len(levels))

def place_box(box, place,i,j,show_plt = False):
    #global W_filp,B_filp,trans_times,i,j,major_b
    is_replace_box = np.clip(copy.deepcopy(place),0,1)
    combina_box = copy.deepcopy(place)

    is_replace_box[j:j+box.shape[0],i:i+box.shape[1]] = is_replace_box[j:j+box.shape[0],i:i+box.shape[1]] + np.clip(copy.deepcopy(box),0,1)
    mask = box >=1
    
    combina_box[j:j+box.shape[0],i:i+box.shape[1]][mask] = box[mask]

    if show_plt:
        fig, ax = plt.subplots(2,2)
        ax[0,0].imshow(place,cmap=item_map,vmin=0, vmax=5.9, interpolation='nearest')
        ax[0,1].imshow(combina_box,cmap=item_map,vmin=0, vmax=5.9, interpolation='nearest')
        ax[1,0].imshow(copy.deepcopy(place),cmap=cmap,vmin=0.1, vmax=4.9)
        ax[1,1].imshow(is_replace_box,cmap=cmap,vmin=0.1, vmax=4.9)
        
        #plt.show()
        plt.savefig('cover check\w{}_b{}_t{}_{}_{}_M{}.jpg'.format(W_filp,B_filp,trans_times,i,j,major_b))
        plt.close()
    return (is_replace_box,combina_box)

#1- production; 2- cheap production; 3- expensive; 4- tech; 5- promot
def record_box(replace_box,Combina_box,n):
    #global W_filp,B_filp,trans_times,i,j,major_b
    record_dict = dict()
    record_dict['n'] = n
    record_dict['replaced'] = (replace_box ==2).sum()
    record_dict['exposed'] = (replace_box ==1).sum()
    
    #result in gameplay #np.isin(arr, values)
    record_dict['production items'] = (Combina_box ==1).sum()
    record_dict['cheap production'] = (Combina_box ==2).sum()
    record_dict['price items'] = (Combina_box ==3).sum()
    record_dict['tech'] = (Combina_box == 4).sum()
    record_dict['promot'] = (Combina_box ==5).sum()
    record_dict['appear items'] = (Combina_box >0).sum()

    record_dict['base production'] = record_dict['production items'] +record_dict['cheap production'] +record_dict['replaced'] 
    record_dict['price'] = 5 + record_dict['price items'] - record_dict['cheap production']

    #placement information
    record_dict['W_side'] = W_filp
    record_dict['B_side'] = B_filp
    record_dict['B_trans_times'] = trans_times
    record_dict['B_placed_i'] = i
    record_dict['B_placed_j'] = j
    record_dict['major_box'] = major_b
    return (record_dict)
'''

box_B[0] = np.array([[2,2,2],[empty_label,2,2]]).transpose()
Combina_box = np.full((7, 7),0)
Combina_box[2:5,2:4]=1
place_box(box_B[0],Combina_box,1,2)'''


C:\Users\LamHouse2020\AppData\Local\Temp\ipykernel_23256\2633464554.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('RdYlGn', 4)  # red, yellow, green, white


'\n\nbox_B[0] = np.array([[2,2,2],[empty_label,2,2]]).transpose()\nCombina_box = np.full((7, 7),0)\nCombina_box[2:5,2:4]=1\nplace_box(box_B[0],Combina_box,1,2)'

In [3]:
empty_label = 0
#create 2 * 2 6 X 6 box
#1- production; 2- cheap production; 3- expensive; 4- tech; 5- promot
box_W = ['','']
box_B = ['','']
box_W[0] = np.array([[5,2,5],[empty_label,4,2]]).transpose()
box_W[1] = np.array([[empty_label,4,1],[4,3,5]]).transpose()
box_B[0] = np.array([[2,5,4],[empty_label,4,1]]).transpose()
box_B[1] = np.array([[empty_label,5,3],[3,5,4]]).transpose()

n = 1
Combina_box_record = dict()
result_df = pd.DataFrame()

#do matching combinations
#flip one, another one, and both
for W_filp in range(2):
    box_W_ues = box_W[W_filp]
    for B_filp in range(2):
#- centered one, and another transposed X3 one placement
        for trans_times in range(4):
            box_B_ues = rot90_t(box_B[B_filp],trans_times)
#- centered one, and another one placement
            for i in range(4):
                for j in range(5):
                    # create 9*8 empty boxes for combination 
                    Combina_box = np.full((7, 7),empty_label)
                    replace_box, Combina_box = place_box(box_W_ues, Combina_box,2,2)
                    replace_box, Combina_box = place_box(box_B_ues, Combina_box,i,j)#,show_plt=True) # show_plt=True if generate display (result in folder "combina box")

                    #by rule, invalid if covered count ==0 or 5
                    if  (replace_box ==2).sum() in [0,5]: 
                        continue
                    major_b = 'b'
                    record_dict = record_box(replace_box,Combina_box,n)
                    result_df = pd.concat([result_df,pd.DataFrame(record_dict,index=[n])])
                    Combina_box_record[n] = Combina_box
                    n=n+1
                    
#- all above but switch covering
                    null, Combina_box = place_box(box_W_ues, Combina_box,2,2)

                    major_b = 'w'
                    record_dict = record_box(replace_box,Combina_box,n)
                    result_df = pd.concat([result_df,pd.DataFrame(record_dict,index=[n])])
                    Combina_box_record[n] = Combina_box
                    n=n+1



In [4]:
# #image generate for check cover (result in folder "cover check")
# for i in Combina_box_record:
#     fig, ax = plt.subplots()
#     ax.imshow(Combina_box_record[i],cmap=item_map,vmin=0, vmax=5.9, interpolation='nearest')
#     plt.savefig('result in cover\{}.jpg'.format(i))
#     plt.close()

In [5]:
# ## Check sandbox
# result_df['check'] = result_df['exposed'] +result_df['replaced'] -result_df['appear items'] 
# result_df['check'].max()

# result_df[['W_side'	,'B_side',	'B_trans_times',	'B_placed_i',	'B_placed_j',	'major_box'	]].describe()
# result_df.groupby(['W_side'	,'B_side',	'B_trans_times',	'B_placed_i',	'B_placed_j',	'major_box'	]).count().max()


In [13]:
#base info
result_df.groupby('price')[['base production','tech','promot']].agg({'base production':['count',max, min],
                                                                     'tech':[max, min],
                                                                     'promot':[max, min],})#.to_csv('value_inPrice.csv')


C:\Users\LamHouse2020\AppData\Local\Temp\ipykernel_23256\776740119.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  result_df.groupby('price')[['base production','tech','promot']].agg({'base production':['count',max, min],
C:\Users\LamHouse2020\AppData\Local\Temp\ipykernel_23256\776740119.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  result_df.groupby('price')[['base production','tech','promot']].agg({'base production':['count',max, min],


base production         tech     promot    
                count max min  max min    max min
price                                            
2                  52   7   4    3   1      3   2
3                  47   6   4    3   2      3   1
4                  65   7   3    4   1      4   1
5                  98   6   3    4   1      4   1
6                  49   6   2    4   1      4   1
7                  61   5   2    3   1      4   1
8                  48   4   1    3   1      3   1

In [7]:

result_df[['price','base production','tech','promot']].sort_values(['price','base production','tech','promot'],ascending=False).to_csv('reviews.csv')

In [8]:
review_df = result_df[['price','base production','tech','promot']]

#any combination always better than other?
ddd = dict()
for i in review_df.index:
    ddd[i] = dict()
    compared_df = (review_df - review_df.loc[i])
    compared_df = compared_df[(compared_df['price']==0) & (compared_df['base production']>=0) &  (compared_df['tech']>=0) &  (compared_df['promot']>=0)]
    ddd[i]['is optimal']= (compared_df.max().sum() ==0)
    if ddd[i]['is optimal'] == False:
        ddd[i]['protenial_increase'] = str(compared_df.max()[compared_df.max()>0].to_dict())
review_df = pd.concat([review_df,pd.DataFrame.from_dict(ddd, orient='index')],axis=1)

#is combination unique
review_df['unique'] = ~review_df[['price','base production','tech','promot']].duplicated(keep = False)

#is first appear combination
review_df['first_sol'] = ~review_df[['price','base production','tech','promot']].duplicated()

#unique & optimal
review_df['unique & optimal'] = review_df['is optimal']*review_df['unique']

review_df


,price,base production,tech,promot,is optimal,protenial_increase,unique,first_sol,unique & optimal
1,2,5,3,2,True,NaN,False,True,False
2,2,4,3,3,True,NaN,False,True,False
3,3,5,3,2,True,NaN,False,True,False
4,2,5,2,3,True,NaN,False,True,False
5,3,5,3,2,True,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...
416,7,3,2,3,True,NaN,False,False,False
417,7,3,3,2,True,NaN,False,False,False
418,7,3,2,3,True,NaN,False,False,False
419,8,2,3,2,True,NaN,False,False,False


In [9]:
review_df[['is optimal','first_sol','unique','unique & optimal']].sum()

is optimal          409
first_sol            58
unique               12
unique & optimal     11
dtype: int64

In [10]:
## display unique optimals combinations
review_df[review_df['unique & optimal']].sort_values(['price','base production','tech','promot'],ascending=False)#.to_csv('unique_optimal.csv')

,price,base production,tech,promot,is optimal,protenial_increase,unique,first_sol,unique & optimal
408,8,4,2,1,True,NaN,True,True,True
358,8,3,3,1,True,NaN,True,True,True
375,8,3,1,3,True,NaN,True,True,True
149,7,3,1,4,True,NaN,True,True,True
278,6,6,2,1,True,NaN,True,True,True
169,6,5,1,2,True,NaN,True,True,True
296,6,4,3,2,True,NaN,True,True,True
277,4,7,2,1,True,NaN,True,True,True
261,4,4,4,2,True,NaN,True,True,True
57,3,6,3,1,True,NaN,True,True,True


In [11]:
##
review_df[~review_df['is optimal']]#.to_csv('not_optimal.csv')

,price,base production,tech,promot,is optimal,protenial_increase,unique,first_sol,unique & optimal
109,6,4,2,2,False,{'tech': 1},True,True,False
120,4,6,1,2,False,{'tech': 1},False,True,False
122,4,5,2,2,False,"{'base production': 1, 'tech': 1}",False,True,False
142,4,5,2,2,False,"{'base production': 1, 'tech': 1}",False,False,False
144,4,6,1,2,False,{'tech': 1},False,False,False
150,4,5,2,2,False,"{'base production': 1, 'tech': 1}",False,False,False
170,4,6,1,2,False,{'tech': 1},False,False,False
172,4,6,1,2,False,{'tech': 1},False,False,False
192,4,5,2,2,False,"{'base production': 1, 'tech': 1}",False,False,False
194,4,5,2,2,False,"{'base production': 1, 'tech': 1}",False,False,False
